In [1]:
import pandas as pd
import requests
import math
import yfinance as yf
from termcolor import colored as cl
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (20, 10)
plt.style.use("fivethirtyeight")

In [2]:
aapl = yf.download("AAPL", start="2015-01-01", end="2020-01-01").reset_index()

/home/jupyter/.cache/pypoetry/virtualenvs/compare-multiple-backtests-2J6ybkfi-py3.10/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


In [3]:
# Calculate the ROI
buy_price = aapl["Close"].iloc[0]
sell_price = aapl["Close"].iloc[-1]

roi = (sell_price - buy_price) / buy_price * 100

print(f"The ROI for a buy and hold strategy is {roi}%")

The ROI for a buy and hold strategy is 168.59049572710646%


In [4]:
counter = 1
for period in [2, 5, 19]:
    aapl["SMA" + str(counter)] = aapl["Close"].rolling(window=period).mean()
    counter += 1

In [5]:
aapl

,Date,Open,High,Low,Close,Adj Close,Volume,SMA1,SMA2,SMA3
0,2015-01-02,27.847500,27.860001,26.837500,27.332500,24.435270,212818400,NaN,NaN,NaN
1,2015-01-05,27.072500,27.162500,26.352501,26.562500,23.746891,257142000,26.947500,NaN,NaN
2,2015-01-06,26.635000,26.857500,26.157499,26.565001,23.749125,263188400,26.563750,NaN,NaN
3,2015-01-07,26.799999,27.049999,26.674999,26.937500,24.082136,160423600,26.751250,NaN,NaN
4,2015-01-08,27.307501,28.037500,27.174999,27.972500,25.007425,237458000,27.455000,27.074000,NaN
...,...,...,...,...,...,...,...,...,...,...
1253,2019-12-24,71.172501,71.222504,70.730003,71.067497,69.147987,48478800,71.033749,70.373499,68.121710
1254,2019-12-26,71.205002,72.495003,71.175003,72.477501,70.519928,93121200,71.772499,70.881999,68.412105
1255,2019-12-27,72.779999,73.492500,72.029999,72.449997,70.493156,146266000,72.463749,71.370999,68.708815
1256,2019-12-30,72.364998,73.172501,71.305000,72.879997,70.911537,144114400,72.664997,71.974998,69.068815


In [11]:
import plotly.graph_objects as go

# Create a trace for each series
trace_close = go.Scatter(
    x=aapl.index[:100],
    y=aapl["Close"][:100],
    mode="lines",
    name="AAPL",
    line=dict(width=5),  # Make line thicker
)

trace_sma1 = go.Scatter(
    x=aapl.index[:100],
    y=aapl["SMA1"][:100],
    mode="lines",
    name="SMA 5",
    line=dict(dash="dot", width=3),  # Make line thicker
)

trace_sma2 = go.Scatter(
    x=aapl.index[:100],
    y=aapl["SMA2"][:100],
    mode="lines",
    name="SMA 8",
    line=dict(dash="dot", width=3),  # Make line thicker
)

trace_sma3 = go.Scatter(
    x=aapl.index[:100],
    y=aapl["SMA3"][:100],
    mode="lines",
    name="SMA 13",
    line=dict(dash="dot", width=3),  # Make line thicker
)

# Create a layout
layout = go.Layout(title="AAPL SMA 5,8,13")

# Create a figure and add traces to it
fig = go.Figure(data=[trace_close, trace_sma1, trace_sma2, trace_sma3], layout=layout)

# Show the figure
fig.show()

In [ ]:
plt.plot(aapl["Close"][:100], label="AAPL", linewidth=6)
plt.plot(aapl["SMA1"][:100], label="SMA 5", linewidth=3, linestyle="--")
plt.plot(aapl["SMA2"][:100], label="SMA 8", linewidth=4, alpha=0.7)
plt.plot(aapl["SMA3"][:100], label="SMA 13", linewidth=4, alpha=0.7)
plt.title("AAPL SMA 5,8,13")
plt.legend()

In [ ]:
# counter = 1
# for period in [5, 8, 13]:
#     aapl["SMA" + str(counter)] = aapl["Close"].rolling(window=period).mean()
#     counter += 1


def implement_strategy(aapl, investment):

    in_position = False
    equity = investment

    for i in range(1, len(aapl)):
        if (
            aapl["SMA2"][i - 1] < aapl["SMA3"][i - 1]
            and aapl["SMA2"][i] > aapl["SMA3"][i]
            and aapl["SMA1"][i] > aapl["SMA2"][i]
            and aapl["Close"][i] > aapl["SMA1"][i]
            and in_position == False
        ):
            no_of_shares = math.floor(equity / aapl.Close[i])
            equity -= no_of_shares * aapl.Close[i]
            in_position = True
            print(
                cl("BUY: ", color="green", attrs=["bold"]),
                f'{no_of_shares} Shares are bought at ${aapl.Close[i]} on {str(aapl["Date"][i])[:10]}',
            )
        elif (
            aapl["SMA2"][i - 1] > aapl["SMA3"][i - 1]
            and aapl["SMA2"][i] < aapl["SMA3"][i]
            and aapl["SMA1"][i] < aapl["SMA2"][i]
            and aapl["Close"][i] < aapl["SMA1"][i]
            and in_position == True
        ):
            equity += no_of_shares * aapl.Close[i]
            in_position = False
            print(
                cl("SELL: ", color="red", attrs=["bold"]),
                f'{no_of_shares} Shares are sold at ${aapl.Close[i]} on {str(aapl["Date"][i])[:10]}',
            )
    if in_position == True:
        equity += no_of_shares * aapl.Close[i]
        print(
            cl(
                f'\nClosing position at {aapl.Close[i]} on {str(aapl["Date"][i])[:10]}',
                attrs=["bold"],
            )
        )
        in_position = False

    earning = round(equity - investment, 2)
    roi = round(earning / investment * 100, 2)
    # return roi
    print(cl(f"EARNING: ${earning} ; ROI: {roi}%", attrs=["bold"]))

In [ ]:
implement_strategy(aapl, 10000)

In [ ]:
from scipy.optimize import brute

# Define ranges for periods
ranges = (slice(1, 50, 1), slice(1, 50, 1), slice(1, 50, 1))


def fitness_func(periods):
    period1, period2, period3 = periods
    # Ensure periods are integers
    period1, period2, period3 = int(period1), int(period2), int(period3)

    # Check if periods meet the constraint
    if not (period1 < period2 < period3):
        return 1e6  # Return a large value if constraint is not met

    # Calculate SMAs
    aapl["SMA1"] = aapl["Close"].rolling(window=period1).mean()
    aapl["SMA2"] = aapl["Close"].rolling(window=period2).mean()
    aapl["SMA3"] = aapl["Close"].rolling(window=period3).mean()

    # Implement strategy and calculate ROI
    roi = implement_strategy(aapl, 10000)
    return -roi  # Negate ROI because brute() minimizes the function


# Perform brute-force optimization
solution = brute(fitness_func, ranges, full_output=True, finish=None)

print("Parameters of the best solution : {solution}".format(solution=solution[0]))